In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import comet_ml
from trl import SFTTrainer
from datasets import load_dataset
from transformers import TrainingArguments, TextStreamer
from unsloth import FastLanguageModel, is_bfloat16_supported
from unsloth.chat_templates import qwen25_template

In [22]:
from news_summarizer.config import settings

In [ ]:
experiment = comet_ml.start(project_name="NewsSummarizerSFT")

model_name = "unsloth/Qwen2.5-1.5B"
max_seq_length = 512
padding_side = "left"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype="float16",
    load_in_4bit=False,
)

tokenizer.chat_template = qwen25_template
tokenizer.padding_side = padding_side
tokenizer.truncation = True

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    lora_alpha=32,
    lora_dropout=0,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    use_gradient_checkpointing="unsloth",  # Save memory
)

### Load data

In [ ]:
dataset = load_dataset(
    "json",
    data_files="../data/dataset/dataset_part*.json",
    split="train",
)


EOS_TOKEN = tokenizer.eos_token


# Format dataset into chat format
def format_to_chat(example):
    return {
        "messages": [
            {"role": "system", "content": "Resuma o seguinte artigo."},
            {"role": "user", "content": example["article"]},
            {"role": "assistant", "content": example["summary"] + EOS_TOKEN},
        ]
    }


# Apply formatting
chat_dataset = dataset.map(format_to_chat, remove_columns=["article", "summary"])

# Split into train and test sets
chat_dataset = chat_dataset.train_test_split(test_size=0.1)

In [ ]:
chat_dataset.push_to_hub(
    "maikerdr/brazilian-news-articles",
    token=settings.huggingface.access_token._secret_value,
)

### Fine tune!

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=4,  # Adjust based on GPU memory
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=3e-4,
    lr_scheduler_type="linear",
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    warmup_steps=10,
    output_dir="../data/outputs",
    seed=0,
    report_to="comet_ml",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=chat_dataset["train"],
    eval_dataset=chat_dataset["test"],
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=training_args,
)

trainer_stats = trainer.train()
experiment.end()

In [ ]:
model.save_pretrained("../data/model/lora_model")  # Local saving
tokenizer.save_pretrained("../data/model/lora_model")

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained("../data/model/lora_model")

In [ ]:
model.push_to_hub(
    "maikerdr/NewsSummarizer-1.5B",
    token=settings.huggingface.access_token._secret_value,
)  # Online saving
tokenizer.push_to_hub(
    "maikerdr/NewsSummarizer-1.5B",
    token=settings.huggingface.access_token._secret_value,
)  # Online saving

In [ ]:
FastLanguageModel.for_inference(model)

In [16]:
messages = [
    {
        "content": "Resuma o seguinte artigo.",
        "role": "system",
    },
    {
        "content": 'Após vencer a disputa pela Presidência da Câmara dos Deputados, Hugo Motta (Republicanos-PB) repetiu gesto de Ulysses Guimarães (1916-1992) ao erguer exemplar da Constituição de 1988 em seu discurso de posse. Eleito no sábado (1º) depois de receber 444 votos dos 513 possíveis, Motta recordou a atuação de Ulysses Guimarães, responsável por promulgar a Constituição Cidadã. "Temos muito o que fazer. Antes de tudo, colegas de trabalho e de vida. Eu sei a responsabilidade que me espera. E sei também que ninguém faz nada sozinho. Citando Ulysses, muitos têm maior probabilidade de acertar do que um só", afirmou o novo presidente da Câmara. Na ocasião, Motta também parafraseou trecho do discurso do presidente da Assembleia Nacional Constituinte em 5 de outubro de 1988 ao declarar que tem "ódio e nojo à ditadura". "Estaremos sempre com a democracia, pela democracia e com a democracia. E seus inimigos encontraram no Legislativo uma barreira como sempre encontraram na história", reiterou o deputado paraibano. "Ainda Estou Aqui" Ao concluir seu discurso de pouco mais de 20 minutos, Motta fez menção ao filme brasileiro "Ainda Estou Aqui", que retrata a história do ex-deputado federal Rubens Paiva (PTB-SP) — desaparecido e morto na ditadura militar. O longa recebeu três indicações ao Oscar 2025. "Temos que estar sempre do lado do Brasil, em harmonia com os demais Poderes. Encerro com uma mensagem de otimismo: ainda estamos aqui", encerrou o parlamentar. Quem é Hugo Motta Aos 35 anos, Hugo Motta Wanderley da Nóbrega tornou-se o deputado mais jovem a assumir o comando da Câmara nos seus quase 200 anos de história. De família tradicional na política paraibana, Motta está em seu quarto mandato consecutivo como deputado federal. Em 2010, foi o parlamentar mais jovem eleito naquele pleito. Motta fez parte da militância política do MDB, partido o qual foi filiado por mais de uma década. Em 2018, no entanto, deixou a sigla para se juntar ao Republicanos, à época chamado de Partido Republicano Brasileiro (PRB). Ao derrotar Marcel van Hattem (Novo-RS) e Pastor Henrique Vieira (PSOL-RJ), Motta levou o Republicanos à Presidência da Câmara pela primeira vez na história do partido',
        "role": "user",
    },
]


text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

In [17]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

In [ ]:
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**model_inputs, streamer=text_streamer, max_new_tokens=512)